# Using autodiff to check gradient/Hessians

In [1]:
using Revise
using DataFrames, Random, GLM, QuasiCopula
using ForwardDiff, LinearAlgebra
using ToeplitzMatrices
using BenchmarkTools
using SnpArrays
using ForwardDiff
# using MendelPlots
ENV["COLUMNS"] = 240

BLAS.set_num_threads(1)
Threads.nthreads()

1

## simulate data

In [5]:
k = 0 # number of causal SNPs
maf = 0.3
p = 3
d = 4

qc_model, G, Btrue, θtrue, γtrue, τtrue = simulate_multivariate_traits(
    n = 5000, # sample size
    p = p, # number of fixed effects, including intercept
    m = 1, # number of variance components
    d = d, # number of phenotypes
    q = 1000, # number of SNPs
    k = k, # number of causal SNPs
    seed = 2023,
    τtrue = 0.01,
    possible_distributions = [Bernoulli, Poisson, Normal]
);

## Is loglikelihood implemented correctly?

To test this, we simulate a single multivariate gaussian Copula, and compare its loglikelihood with longitudinal gaussian copula with only 1 observation per sample

In [45]:
n = 5000

qc_model1, G, Btrue, θtrue, γtrue, τtrue = simulate_multivariate_traits(
    n = n, # sample size
    p = 3, # number of fixed effects, including intercept
    m = 1, # number of variance components
    d = 1, # number of phenotypes
    q = 1000, # number of SNPs
    k = k, # number of causal SNPs
    seed = 2023,
    τtrue = 0.01,
    possible_distributions = [Normal]
)

qc_model2, G, Btrue, θtrue, γtrue, τtrue = simulate_longitudinal_traits(
    n = n, # sample size
    p = 3, # number of fixed effects, including intercept
    m = 1, # number of variance components
    d_max = 1, # number of observations per sample
    q = 1000, # number of SNPs
    k = k, # number of causal SNPs
    seed = 2023,
    τtrue = 0.01,
    y_distribution = Normal,
)

# force longitudinal qc_model to share the same y/X/θ/τ as multivariate case
for i in 1:n
    copyto!(qc_model2.data[i].y, qc_model1.Y[i, :])
    copyto!(qc_model2.data[i].X, qc_model1.X[i, :])
end
qc_model2.θ .= qc_model1.θ
qc_model2.τ .= qc_model1.ϕ

1-element Vector{Float64}:
 1.0

In [46]:
[qc_model1.Y vcat([qc_model2.data[i].y for i in 1:5000]...)]

5000×2 Matrix{Float64}:
  -4.35162   -4.35162
  -4.87728   -4.87728
   1.892      1.892
  -8.44168   -8.44168
   5.00282    5.00282
  -6.28996   -6.28996
   2.48012    2.48012
   5.39039    5.39039
  20.9844    20.9844
  -5.22089   -5.22089
 -20.0763   -20.0763
 -14.0114   -14.0114
  -5.6128    -5.6128
   ⋮        
  -2.81213   -2.81213
  -2.18985   -2.18985
  -2.28793   -2.28793
  -2.78477   -2.78477
   2.93928    2.93928
  10.5193    10.5193
   7.13238    7.13238
 -15.9023   -15.9023
 -24.1438   -24.1438
   2.52053    2.52053
  -1.52918   -1.52918
 -12.7906   -12.7906

In [49]:
@show loglikelihood!(qc_model1, false, false)
@show loglikelihood!(qc_model2, false, false);

loglikelihood!(qc_model1, false, false) = -265911.4748951471
loglikelihood!(qc_model2, false, false) = -270985.42927297653


## Is $\nabla_\beta res$ calculated correctly? 

We can check using ForwardDiff

The function is 

$$res_{ij}(\beta) = \frac{y_{ij} - \mu_{ij}}{\sqrt{\sigma_{ij}^2(\beta)}}$$

### Normal

Assumes y, X are given. We calculate the (standardized) residuals for just 1 sample

In [6]:
# sample data
xi = qc_model.X[1, :] # p by 1
yi = qc_model.Y[1, :] # d by 1

# objective
function resβ(y, x, vecB::AbstractVector)
    p = length(x)
    d = length(y)
    B = reshape(vecB, p, d)
    η = B' * x
    μ = GLM.linkinv.(IdentityLink(), η)
    varμ = GLM.glmvar.(Normal(), μ)
    return (y - μ) ./ sqrt.(varμ)
end
resβ(B::AbstractMatrix) = resβ(yi, xi, vec(B))
resβ(B::AbstractVector) = resβ(yi, xi, B)

B = randn(p, d)
@show resβ(yi, xi, vec(B))

# autodiff gradient
∇resβ_autodiff = x -> ForwardDiff.jacobian(resβ, x)
correct = ∇resβ_autodiff(vec(B))'

# mathematical gradient
function ∇resβ(x, y, vecB::AbstractVector{T}) where T
    p = length(x)
    d = length(y)
    B = reshape(vecB, p, d)
    η = B' * x
    μ = GLM.linkinv.(IdentityLink(), η)
    varμ = GLM.glmvar.(Normal(), μ)
    res = y - μ
    std_res = res ./ sqrt.(varμ)
    dμ = GLM.mueta.(IdentityLink(), η)
    out = zeros(T, d*p, d)
    @inbounds for j in 1:d
        for k in 1:p
            out[(j-1)*p + k, j] = QuasiCopula.update_∇res_ij(Normal(), x[k], 
                std_res[j], μ[j], dμ[j], varμ[j])
        end
    end
    return out
end
math_result = ∇resβ(xi, yi, vec(B))

# compare results
[vec(math_result) vec(correct)]

resβ(yi, xi, vec(B)) = [1.3884090998930962, -0.16822204638920013, 2.241041968786522, 0.04559962991400112]


48×2 Matrix{Float64}:
 -1.0        -1.0
 -0.100418   -0.100418
  0.0470154   0.0470154
  0.0        -0.0
  0.0        -0.0
  0.0        -0.0
  0.0        -0.0
  0.0        -0.0
  0.0        -0.0
  0.0        -0.0
  0.0        -0.0
  0.0        -0.0
  0.0         0.0
  ⋮          
  0.0        -0.0
  0.0        -0.0
  0.0        -0.0
  0.0        -0.0
  0.0        -0.0
  0.0        -0.0
  0.0        -0.0
  0.0        -0.0
  0.0        -0.0
 -1.0        -1.0
 -0.100418   -0.100418
  0.0470154   0.0470154

### Bernoulli

In [6]:
# sample data
xi = qc_model.X[1, :] # p by 1
yi = qc_model.Y[1, :] # d by 1

# objective
function resβ(y, x, vecB::AbstractVector)
    p = length(x)
    d = length(y)
    B = reshape(vecB, p, d)
    η = B' * x
    μ = GLM.linkinv.(LogitLink(), η)
    varμ = GLM.glmvar.(Bernoulli(), μ)
    return (y - μ) ./ sqrt.(varμ)
end
resβ(B::AbstractMatrix) = resβ(yi, xi, vec(B))
resβ(B::AbstractVector) = resβ(yi, xi, B)

B = randn(p, d)
@show resβ(yi, xi, vec(B))

# autodiff gradient
∇resβ_autodiff = x -> ForwardDiff.jacobian(resβ, x)
correct = ∇resβ_autodiff(vec(B))'

# mathematical gradient
function ∇resβ(x, y, vecB::AbstractVector{T}) where T
    p = length(x)
    d = length(y)
    B = reshape(vecB, p, d)
    η = B' * x
    μ = GLM.linkinv.(LogitLink(), η)
    varμ = GLM.glmvar.(Bernoulli(), μ)
    res = y - μ
    std_res = res ./ sqrt.(varμ)
    dμ = GLM.mueta.(LogitLink(), η)
    out = zeros(T, d*p, d)
    @inbounds for j in 1:d
        for k in 1:p
            out[(j-1)*p + k, j] = QuasiCopula.update_∇res_ij(Bernoulli(), x[k], 
                std_res[j], μ[j], dμ[j], varμ[j])
        end
    end
    return out
end
math_result = ∇resβ(xi, yi, vec(B))

# compare results
[vec(math_result) vec(correct)]

resβ(yi, xi, vec(B)) = [-1.699553635820773, -2.9658266436544203, 2.621786668608018]


45×2 Matrix{Float64}:
 -0.849777  -0.849777
  0.262282   0.262282
 -1.92764   -1.92764
 -0.268584  -0.268584
 -0.564345  -0.564345
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  ⋮         
  0.0       -0.0
  0.0       -0.0
  0.0       -0.0
  0.0       -0.0
  0.0       -0.0
  0.0       -0.0
  0.0       -0.0
 -1.31089   -1.31089
  0.404604   0.404604
 -2.97365   -2.97365
 -0.414327  -0.414327
 -0.870577  -0.870577

### Poisson

In [7]:
# sample data
xi = qc_model.X[1, :] # p by 1
yi = qc_model.Y[1, :] # d by 1

# objective
function resβ(y, x, vecB::AbstractVector)
    p = length(x)
    d = length(y)
    B = reshape(vecB, p, d)
    η = B' * x
    μ = GLM.linkinv.(LogLink(), η)
    varμ = GLM.glmvar.(Poisson(), μ)
    return (y - μ) ./ sqrt.(varμ)
end
resβ(B::AbstractMatrix) = resβ(yi, xi, vec(B))
resβ(B::AbstractVector) = resβ(yi, xi, B)

B = randn(p, d)
@show resβ(yi, xi, vec(B))

# autodiff gradient
∇resβ_autodiff = x -> ForwardDiff.jacobian(resβ, x)
correct = ∇resβ_autodiff(vec(B))'

# mathematical gradient
function ∇resβ(x, y, vecB::AbstractVector{T}) where T
    p = length(x)
    d = length(y)
    B = reshape(vecB, p, d)
    η = B' * x
    μ = GLM.linkinv.(LogLink(), η)
    varμ = GLM.glmvar.(Poisson(), μ)
    res = y - μ
    std_res = res ./ sqrt.(varμ)
    dμ = GLM.mueta.(LogLink(), η)
    out = zeros(T, d*p, d)
    @inbounds for j in 1:d
        for k in 1:p
            out[(j-1)*p + k, j] = QuasiCopula.update_∇res_ij(Poisson(), x[k], 
                std_res[j], μ[j], dμ[j], varμ[j])
        end
    end
    return out
end
math_result = ∇resβ(xi, yi, vec(B))

# compare results
[vec(math_result) vec(correct)]

resβ(yi, xi, vec(B)) = [-0.6311261621867231, -2.784614111314093, 18.689464943875688]


45×2 Matrix{Float64}:
  -0.315563    -0.315563
   0.0973979    0.0973979
  -0.715827    -0.715827
  -0.0997383   -0.0997383
  -0.209568    -0.209568
   0.0          0.0
   0.0          0.0
   0.0          0.0
   0.0          0.0
   0.0          0.0
   0.0          0.0
   0.0          0.0
   0.0          0.0
   ⋮          
   0.0         -0.0
   0.0         -0.0
   0.0         -0.0
   0.0         -0.0
   0.0         -0.0
   0.0         -0.0
   0.0         -0.0
  -9.39809     -9.39809
   2.9007       2.9007
 -21.3187     -21.3187
  -2.9704      -2.9704
  -6.24136     -6.24136

## Check $\nabla_{\beta} L$

In [24]:
# sample data
i = 10
xi = qc_model.X[i, :] # p by 1
yi = qc_model.Y[i, :] # d by 1

function A_mul_b!(c::AbstractVector{T}, A::AbstractMatrix, b::AbstractVector) where T
    n, p = size(A)
    fill!(c, zero(T))
    for j in 1:p, i in 1:n
        c[i] += A[i, j] * b[j]
    end
    return c
end

# loglikelihood for a single sample
function loglikelihood(y, x, vecB::AbstractVector{T}, qc_model) where T
    p = length(x)
    d = length(y)
    m = qc_model.m
    B = reshape(vecB, p, d)
    η = B' * x
    μ = GLM.linkinv.(qc_model.veclink, η)
    varμ = GLM.glmvar.(qc_model.vecdist, μ)
    res = y - μ
    std_res = res ./ sqrt.(varμ)
    dμ = GLM.mueta.(LogLink(), η)
    storage_d = zeros(T, d)
    q = zeros(T, m)
    # GLM loglikelihood (term 2)
    logl = zero(T)
    @inbounds for j in eachindex(y)
        logl += QuasiCopula.loglik_obs(qc_model.vecdist[j], y[j], μ[j], one(T), one(T))
    end
    # loglikelihood term 1 i.e. -sum ln(1 + 0.5tr(Γ(θ)))
    tsum = dot(qc_model.θ, qc_model.t) # tsum = 0.5tr(Γ)
    logl += -log(1 + tsum)
    #
     
    # loglikelihood term 3 i.e. sum ln(1 + 0.5 r*Γ*r)
    @inbounds for k in 1:qc_model.m # loop over m variance components
        mul!(storage_d, qc_model.V[k], std_res) # storage_d = V[k] * r
#         if needgrad
#             BLAS.gemv!('N', θ[k], qc.∇resβ, qc.storage_d, one(T), qc.∇vecB) # ∇β = ∇r*Γ*r
#         end
        q[k] = dot(std_res, storage_d) / 2 # q[k] = 0.5 r * V[k] * r
    end
    qsum = dot(qc_model.θ, q) # qsum = 0.5 r*Γ*r
    logl += log(1 + qsum)
    return logl
end
loglikelihood(B::AbstractMatrix) = loglikelihood(yi, xi, vec(B), qc_model)
loglikelihood(B::AbstractVector) = loglikelihood(yi, xi, B, qc_model)

# autodiff gradient
logl_autodiff = x -> ForwardDiff.gradient(loglikelihood, x)
correct = logl_autodiff(vec(B))

# gradient from math
qc_model.B .= B
loglikelihood!(qc_model, true, false)

[correct qc_model.data[i].∇vecB]

15×2 Matrix{Float64}:
  0.472462    0.472462
  0.325011    0.325011
  0.314195    0.314195
  0.0177468   0.0177468
  0.565479    0.565479
 -0.958411   -0.958411
 -0.6593     -0.6593
 -0.637359   -0.637359
 -0.0360001  -0.0360001
 -1.1471     -1.1471
  4.56203     4.56203
  3.13827     3.13827
  3.03382     3.03382
  0.171361    0.171361
  5.4602      5.4602

## Check $\nabla_{\theta} L$

In [13]:
# sample data
i = 5
xi = qc_model.X[i, :] # p by 1
yi = qc_model.Y[i, :] # d by 1

function A_mul_b!(c::AbstractVector{T}, A::AbstractMatrix, b::AbstractVector) where T
    n, p = size(A)
    fill!(c, zero(T))
    for j in 1:p, i in 1:n
        c[i] += A[i, j] * b[j]
    end
    return c
end

# loglikelihood for a single sample
function loglikelihood(y, x, θ::AbstractVector{T}, qc_model) where T
    p = length(x)
    d = length(y)
    m = qc_model.m
    B = qc_model.B
    η = B' * x
    μ = GLM.linkinv.(qc_model.veclink, η)
    varμ = GLM.glmvar.(qc_model.vecdist, μ)
    res = y - μ
    std_res = res ./ sqrt.(varμ)
    storage_d = zeros(T, d)
    q = zeros(T, m)
    # GLM loglikelihood (term 2)
    logl = zero(T)
    @inbounds for j in eachindex(y)
        logl += QuasiCopula.loglik_obs(qc_model.vecdist[j], y[j], μ[j], one(T), one(T))
    end
    # loglikelihood term 1 i.e. -sum ln(1 + 0.5tr(Γ(θ)))
    tsum = dot(θ, qc_model.t) # tsum = 0.5tr(Γ)
    logl += -log(1 + tsum)
    # loglikelihood term 3 i.e. sum ln(1 + 0.5 r*Γ*r)
    @inbounds for k in 1:qc_model.m # loop over m variance components
        mul!(storage_d, qc_model.V[k], std_res) # storage_d = V[k] * r
#         if needgrad
#             BLAS.gemv!('N', θ[k], qc.∇resβ, qc.storage_d, one(T), qc.∇vecB) # ∇β = ∇r*Γ*r
#         end
        q[k] = dot(std_res, storage_d) / 2 # q[k] = 0.5 r * V[k] * r
    end
    qsum = dot(θ, q) # qsum = 0.5 r*Γ*r
    logl += log(1 + qsum)
    return logl
end
loglikelihood(θ::AbstractVector) = loglikelihood(yi, xi, θ, qc_model)

# autodiff gradient
logl_autodiff = x -> ForwardDiff.gradient(loglikelihood, x)

θ = rand(2)
correct = logl_autodiff(θ)

# gradient from math
qc_model.θ .= θ
loglikelihood!(qc_model, true, false)

[correct qc_model.data[i].∇θ]

2×2 Matrix{Float64}:
 -0.60808   -0.60808
 -0.496524  -0.496524

## Check $\nabla_{\phi} L$

In [ ]:
# sample data
i = 5
xi = qc_model.X[i, :] # p by 1
yi = qc_model.Y[i, :] # d by 1

function A_mul_b!(c::AbstractVector{T}, A::AbstractMatrix, b::AbstractVector) where T
    n, p = size(A)
    fill!(c, zero(T))
    for j in 1:p, i in 1:n
        c[i] += A[i, j] * b[j]
    end
    return c
end

# loglikelihood for a single sample
function loglikelihood(y, x, ϕ::AbstractVector{T}, qc_model) where T
    p = length(x)
    d = length(y)
    m = qc_model.m
    B = qc_model.B
    η = B' * x
    μ = GLM.linkinv.(qc_model.veclink, η)
    varμ = GLM.glmvar.(qc_model.vecdist, μ)
    res = y - μ
    std_res = T[]
    nuisance_counter = 1
    for j in eachindex(res)
        if typeof(qc_model.vecdist[j]) <: Normal
            τ = abs(qc_model.ϕ[nuisance_counter])
            push!(std_re, res * sqrt(τ))
            nuisance_counter += 1
        else
            push!(std_res, res / sqrt(varμ[j]))
        end
    end
#     std_res = res ./ sqrt.(varμ)
    storage_d = zeros(T, d)
    q = zeros(T, m)
    # GLM loglikelihood (term 2)
    logl = zero(T)
    @inbounds for j in eachindex(y)
        logl += QuasiCopula.loglik_obs(qc_model.vecdist[j], y[j], μ[j], one(T), one(T))
    end
    # loglikelihood term 1 i.e. -sum ln(1 + 0.5tr(Γ(θ)))
    tsum = dot(θ, qc_model.t) # tsum = 0.5tr(Γ)
    logl += -log(1 + tsum)
    # also add contributions from nuisance parameters
    for (j, idx) in enumerate(qc_model.nuisance_idx)
        τ = abs(qc_model.ϕ[j])
        rss = abs2(norm(qc.std_res[idx]))
        logl += -(1 * log(2π) - 1 * log(abs(τ)) + rss) / 2
    end
    # loglikelihood term 3 i.e. sum ln(1 + 0.5 r*Γ*r)
    @inbounds for k in 1:qc_model.m # loop over m variance components
        mul!(storage_d, qc_model.V[k], std_res) # storage_d = V[k] * r
#         if needgrad
#             BLAS.gemv!('N', θ[k], qc.∇resβ, qc.storage_d, one(T), qc.∇vecB) # ∇β = ∇r*Γ*r
#         end
        q[k] = dot(std_res, storage_d) / 2 # q[k] = 0.5 r * V[k] * r
    end
    qsum = dot(θ, q) # qsum = 0.5 r*Γ*r
    logl += log(1 + qsum)
    return logl
end
loglikelihood(ϕ::AbstractVector) = loglikelihood(yi, xi, ϕ, qc_model)

# autodiff gradient
logl_autodiff = x -> ForwardDiff.gradient(loglikelihood, x)

θ = rand(2)
correct = logl_autodiff(θ)

# gradient from math
qc_model.θ .= θ
loglikelihood!(qc_model, true, false)

[correct qc_model.data[i].∇θ]